<a href="https://colab.research.google.com/github/Qoonitah18/Project-1/blob/main/Task_3_Data_Analyst_Eklipse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# 1. MEMUAT DATA & KLASIFIKASI PENGGUNA
# SQL: SELECT * FROM table

df_gs = pd.read_csv("gamesession.csv")
df_prem = pd.read_csv("premium.csv")
df_dl = pd.read_csv("downloaded_clips.csv")
df_clips = pd.read_csv("clips.csv")
df_share = pd.read_csv("shared_clips.csv")

In [ ]:
# Perbaikan ID & Konversi Dasar
df_dl.rename(columns={'user_Id': 'user_id'}, inplace=True)
df_gs['duration'] = pd.to_numeric(df_gs['duration'], errors='coerce') / 60000 # Ubah milisecond ke menit

In [ ]:
# Pembagian Jenis Pengguna
premium_users = df_prem['user_id'].unique()
all_active_users = df_gs['user_id'].unique()

df_users = pd.DataFrame({'user_id': all_active_users})
df_users['user_type'] = np.where(df_users['user_id'].isin(premium_users), 'Premium', 'Reguler')

In [ ]:
# Hitung Total Pengguna
user_summary = df_users.groupby('user_type').size().reset_index(name='total_users')

In [ ]:
print("## Hasil Langkah 1: Ringkasan Basis Pengguna")
print(user_summary.to_markdown(index=False))

## Hasil Langkah 1: Ringkasan Basis Pengguna
| user_type   |   total_users |
|:------------|--------------:|
| Premium     |           333 |
| Reguler     |          3989 |


In [ ]:
# 2. METRIK PERILAKU PENGGUNA
# SQL: JOIN, GROUP BY, AVG

# A. Rata-rata Durasi Sesi Game (Menit)
# JOIN gamesession DENGAN user_groups, lalu AVG(duration) GROUP BY user_type
df_gs_merged = df_gs.merge(df_users, on='user_id', how='inner')
avg_duration = df_gs_merged.groupby('user_type')['duration'].mean().reset_index(name='avg_session_duration_min').round(2)

In [ ]:
# B. Rata-rata Unduhan Klip per Pengguna (Paling Penting)
# Hitung COUNT(downloads) per user, JOIN DENGAN user_groups, lalu AVG(COUNT) GROUP BY user_type
downloads_per_user = df_dl.groupby('user_id').size().reset_index(name='download_count')
df_dl_merged = df_users.merge(downloads_per_user, on='user_id', how='left').fillna(0)
avg_downloads = df_dl_merged.groupby('user_type')['download_count'].mean().reset_index(name='avg_downloads_per_user').round(2)

In [ ]:
print("## Hasil Langkah 2: Durasi dan Unduhan Klip")
print("Rata-rata Durasi Sesi (Menit):")
print(avg_duration.to_markdown(index=False))
print("\nRata-rata Unduhan Klip per Pengguna:")
print(avg_downloads.to_markdown(index=False))

## Hasil Langkah 2: Durasi dan Unduhan Klip
Rata-rata Durasi Sesi (Menit):
| user_type   |   avg_session_duration_min |
|:------------|---------------------------:|
| Premium     |                     342.16 |
| Reguler     |                     344.5  |

Rata-rata Unduhan Klip per Pengguna:
| user_type   |   avg_downloads_per_user |
|:------------|-------------------------:|
| Premium     |                     3.41 |
| Reguler     |                     2.96 |


In [ ]:
# 3. METRIK CHURN RATE
df_prem['canceled_at'] = pd.to_datetime(df_prem['canceled_at'], errors='coerce')
churn_rate = (df_prem['canceled_at'].count() / len(df_prem)) * 100

In [ ]:
# 4. HASIL AKHIR
user_summary = df_users.groupby('user_type').size().reset_index(name='total_users')
df_final = user_summary.merge(avg_duration, on='user_type', how='left')
df_final = df_final.merge(avg_downloads, on='user_type', how='left')

print("### TABEL RINGKASAN ###")
print(df_final.to_markdown(index=False))
print(f"\nTingkat Pembatalan (Churn Rate) Premium: {churn_rate:.2f}%")

### TABEL RINGKASAN ###
| user_type   |   total_users |   avg_session_duration_min |   avg_downloads_per_user |
|:------------|--------------:|---------------------------:|-------------------------:|
| Premium     |           333 |                     342.16 |                     3.41 |
| Reguler     |          3989 |                     344.5  |                     2.96 |

Tingkat Pembatalan (Churn Rate) Premium: 32.55%
